In [12]:
import keras
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_auc_score

class_names = ["confirmed", "crossedout", "empty"]

In [3]:
model = keras.saving.load_model("models/omr_model.keras")
test_ds =  tf.data.Dataset.load("datasets/ds_test")

In [4]:
true_labels = []
for _, label in test_ds:
    true_labels.append(label.numpy())

true_labels = np.concatenate(true_labels)

In [5]:
results = model.predict(test_ds)

53/53 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step


In [6]:
predicted_labels = np.argmax(results, axis=1)

In [7]:
_, predicted_ammount = np.unique(predicted_labels, return_counts=True)

In [8]:
precision, recall, fscore, support = precision_recall_fscore_support(true_labels, predicted_labels, average=None)

In [9]:
auc = roc_auc_score(true_labels, results, average=None, multi_class='ovr')

In [20]:
acc = accuracy_score(true_labels, predicted_labels) * 100

print(f"Test dataset accuracy: {acc:.2f}%")

Test dataset accuracy: 97.17%


In [10]:
import pandas as pd

pd.DataFrame(
    {"class": class_names, "support": support, "predicted_ammount": predicted_ammount, "precision": precision, "recall": recall, "fscore": fscore, "auc": auc}
)

,class,support,predicted_ammount,precision,recall,fscore,auc
0,confirmed,4463,4305,0.985598,0.950706,0.967838,0.993800
1,crossedout,80,327,0.171254,0.700000,0.275184,0.920052
2,empty,8876,8787,0.994651,0.984678,0.989639,0.998756


In [18]:
cm = confusion_matrix(true_labels, predicted_labels)

df = pd.DataFrame(cm, index=class_names, columns=class_names)
df.index.name = "True"
df.columns.name = "Predicted"
df

Predicted,confirmed,crossedout,empty
True,,,
confirmed,4243,179,41
crossedout,18,56,6
empty,44,92,8740
